# Jupyter Notebook Dispatcher based on metadata content

### Execute the cells below to get parameters from metadata and select a notebook for processing ###


First cell executes some Javascript.

the execute command defines the variables documentID, user, and full_notebook_url used later in python

The Javascript function getQueryString takes 'key' argument that is a string parameter name. The calling URL that opens the notebook will return the value assigned to that URL parameter if the parameter is present. 
example URL: http://suave-jupyterhub.com/user/zeppelin-v/notebooks/CinergiDispatch-UseMetadata.ipynb?documentId=f8617294d50d494dae64d8286fb2efaa

http://{jupypter hub host}/user/{user registered with the hub}/notebooks/{name of this notebook}?{parameter1}={value1}

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("documentID='".concat(getQueryStringValue("documentId")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 



## Parameters passed to workbench: ##

In [ ]:
# Check if the parameters are correct
import ntpath
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import webbrowser
url_partitioned = full_notebook_url.partition('/CinergiDispatch')
base_url = url_partitioned[0];

# echo variables for clarity
print("User: ",user)
print("DocumentID: ", documentID)
print("full notebook url partition", url_partitioned)
print("full notebook url", full_notebook_url)

#ToDo: 
1. get the metadata record for the URL; probably have to call ESRI JSON because of uncertainty about what dialect the XML will use
2. extract distribution and format information to use for filtering the action options
3. offer user choice of workbench actions
4. execute action; this will likely be a system command or opening another URL that might be another notebook or some other web application


In [1]:
import json

import requests
from lxml import etree  #supposed to be better than xml.etree
import sys
#import xml.etree.ElementTree as ET
from io import StringIO,BytesIO
from owslib.wfs import WebFeatureService
from shapely.geometry import Polygon, mapping, asShape, shape

catalogURL = "http://cinergi.sdsc.edu/geoportal/"
#documentID="e3619c5df2644204b67f51f48525a0b1"
documentID="4db8156abb6d4119aa5c35aa39514b42"

metadataURLJ=catalogURL + 'rest/metadata/item/' + documentID

print ("metadata URL: ", metadataURLJ)

the_page = requests.get(metadataURLJ)

thejson=json.loads(the_page.text)

for resource in thejson["_source"]["resources_nst"]:
    #print (resource["url_type_s"])
    if resource["url_type_s"] == 'WFS':
        #print("WFS link: ",resource["url_s"])
        resourceurl=resource["url_s"]
        
        url_partitioned = resourceurl.partition('?')
        base_url = url_partitioned[0];
        print("base URL: ",base_url)
        wfs = WebFeatureService(base_url, version='1.1.0')
        
        break
        
if not (wfs):
    print ("no wfs found")
    sys.exit()

print (list(wfs.contents))

metadata URL:  http://cinergi.sdsc.edu/geoportal/rest/metadata/item/4db8156abb6d4119aa5c35aa39514b42
base URL:  https://www.sciencebase.gov/catalogMaps/mapping/ows/5032ab9de4b0d64661a77224
['sb:footprint', 'sb:WYSagegrouse_currentdistribution']


In [3]:
# !pip install osgeo


  Could not find a version that satisfies the requirement osgeo (from versions: )
No matching distribution found for osgeo
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
try:
    from osgeo import ogr, osr, gdal
except:
    sys.exit('ERROR: cannot find GDAL/OGR modules')

#get the url to retrieve xml record from catalog
metadataURLx=catalogURL + 'rest/metadata/item/' + documentID + '.xml'

print ("metadata URL: ", metadataURLx)

#get the xml record
the_page = requests.get(metadataURLx)



SystemExit: ERROR: cannot find GDAL/OGR modules

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#covert to json
the_page_json=thepage.ExportToJson()